In [ ]:
import argparse
import pysam
from pathlib import Path
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
from scipy.stats import fisher_exact
import scipy.stats as stats
import os
import pandas as pd

In [ ]:
#add the CHANGE seq beds
alt7_taylor = 'alt7_taylor.bed'
rex4_taylor = 'rex4_taylor.bed'
rexc_taylor = 'rexc_taylor.bed'
altc_taylor = 'altc_taylor.bed'

#the cas offinder beds 
alt7_cof =  Path('small_bams_again') / 'beds' / 'casoffinder_beds' / 'casoffinder_alt7_4mismatches.bed'
rex4_cof =  Path('small_bams_again') / 'beds' / 'casoffinder_beds' / 'casoffinder_rex4_4mismatches.bed'
rexc_cof =  Path('small_bams_again') / 'beds' / 'casoffinder_beds' / 'casoffinder_rexc_4mismatches.bed'
altc_cof =  Path('small_bams_again') / 'beds' / 'casoffinder_beds' / 'casoffinder_altc_4mismatches.bed'


altc = Path('small_bams_again') / 'beds' / 'altc_lifted.bed'
alt7 = Path('small_bams_again') / 'beds' / 'alt7_lifted.bed'
rex4 = Path('small_bams_again') / 'beds' / 'rex4_lifted.bed'
rexc = Path('small_bams_again') / 'beds' / 'rexc_lifted.bed'

altc = 'altc_taylor.bed'
alt7 = 'alt7_taylor.bed'
rex4 = 'rex4_taylor.bed'
rexc = 'rexc_taylor.bed'

#read in the tsvs
all_tsvs = {'ALTC':pd.read_csv(altc, sep='\t',  names=['chrom','start','end','na','na2','REGION']),  
    'ALT7':pd.read_csv(alt7, sep='\t',  names=['chrom','start','end','na','na2','REGION']),
                'REX4':pd.read_csv(rex4, sep='\t',  names=['chrom','start','end','na','na2','REGION']),
                'REXC':pd.read_csv(rexc, sep='\t',  names=['chrom','start','end','na','na2','REGION'])}


all_tsvs_cof = {'ALTC':pd.read_csv(altc_cof, sep='\t',  names=['chrom','start','end','na','na2','REGION']),  
    'ALT7':pd.read_csv(alt7_cof, sep='\t',  names=['chrom','start','end','na','na2','REGION']),
                'REX4':pd.read_csv(rex4_cof, sep='\t',  names=['chrom','start','end','na','na2','REGION']),
                'REXC':pd.read_csv(rexc_cof, sep='\t',  names=['chrom','start','end','na','na2','REGION'])}



edited_dfs = {'ALTC': [None, None],'ALT7': [None, None],
      'REXC': [None, None],
      'REX4':[None, None]}

comp_dfs = {'ALTC': [None, None], 
            'ALT7': [None, None],
      'REXC': [None, None],
      'REX4':[None, None]}

control_dfs = {'ALTC': [None, None],
               'ALT7': None,
      'REXC': None,
      'REX4': None}

edited_dfs_cof = {'ALTC': [None, None],'ALT7': [None, None],
      'REXC': [None, None],
      'REX4':[None, None]}

comp_dfs_cof = {'ALTC': [None, None], 
            'ALT7': [None, None],
      'REXC': [None, None],
      'REX4':[None, None]}

control_dfs_cof = {'ALTC': [None, None],
               'ALT7': None,
      'REXC': None,
      'REX4': None}


#these are the cassofinder csvs
for condition in all_tsvs:
    for replicate in  np.arange(0,2):
        file_path = f"{condition}again_casoffinder_new_concatanted_taylor_consensus_indel_rates_hg38{replicate}.csv"  # Construct file name
        edited_dfs_cof[condition][replicate] = pd.read_csv(file_path)
        edited_dfs_cof[condition][replicate]['chrom_orig'] = all_tsvs_cof[condition]['chrom'].to_list()
        edited_dfs_cof[condition][replicate]['start_orig'] = all_tsvs_cof[condition]['start'].to_list()
        edited_dfs_cof[condition][replicate]['end_orig'] = all_tsvs_cof[condition]['end'].to_list()
        
for condition in all_tsvs:
    file_path = f"{condition}again_casoffinder_new_concatanted_taylor_consensus_indel_rates_control_hg38.csv"  # Using f-string for readability
    control_dfs_cof[condition] = pd.read_csv(file_path)
    control_dfs_cof[condition]['chrom_orig'] = all_tsvs_cof[condition]['chrom'].to_list()
    control_dfs_cof[condition]['start_orig'] = all_tsvs_cof[condition]['start'].to_list()
    control_dfs_cof[condition]['end_orig'] = all_tsvs_cof[condition]['end'].to_list()

for condition in all_tsvs:
    for replicate in np.arange(0,2):
        file_path = f"casoffinder_{condition}{replicate}_taylor_consensus_comparison_hg38.csv"  # Using f-string for readability
        comp_dfs_cof[condition][replicate] = pd.read_csv(file_path)
        comp_dfs_cof[condition][replicate]['chrom_orig'] = all_tsvs_cof[condition]['chrom'].to_list()
        comp_dfs_cof[condition][replicate]['start_orig'] = all_tsvs_cof[condition]['start'].to_list()
        comp_dfs_cof[condition][replicate]['end_orig'] = all_tsvs_cof[condition]['end'].to_list()

#these are the normal ones
for condition in all_tsvs:
    for replicate in  np.arange(0,2):
        file_path = f"{condition}again_new_concatanted_taylor_consensus_indel_rates_hg38{replicate}.csv"  # Construct file name
        edited_dfs[condition][replicate] = pd.read_csv(file_path)
        edited_dfs[condition][replicate]['chrom_orig'] = all_tsvs[condition]['chrom'].to_list()
        edited_dfs[condition][replicate]['start_orig'] = all_tsvs[condition]['start'].to_list()
        edited_dfs[condition][replicate]['end_orig'] = all_tsvs[condition]['end'].to_list()

for condition in all_tsvs:
    file_path = f"{condition}again_new_concatanted_taylor_consensus_indel_rates_control_hg38.csv"  # Using f-string for readability
    control_dfs[condition] = pd.read_csv(file_path)
    control_dfs[condition]['chrom_orig'] = all_tsvs[condition]['chrom'].to_list()
    control_dfs[condition]['start_orig'] = all_tsvs[condition]['start'].to_list()
    control_dfs[condition]['end_orig'] = all_tsvs[condition]['end'].to_list()

for condition in all_tsvs:
    for replicate in np.arange(0,2):
        file_path = f"change_seq_{condition}{replicate}_taylor_consensus_comparison_hg38.csv"  # Using f-string for readability
        print(file_path)
        comp_dfs[condition][replicate] = pd.read_csv(file_path)
        comp_dfs[condition][replicate]['chrom_orig'] = all_tsvs[condition]['chrom'].to_list()
        comp_dfs[condition][replicate]['start_orig'] = all_tsvs[condition]['start'].to_list()
        comp_dfs[condition][replicate]['end_orig'] = all_tsvs[condition]['end'].to_list()


In [ ]:
from scipy.stats import combine_pvalues
comp_dfs['ALTC'][0]

#combine the graphs
for condition in all_tsvs.keys():
    print(comp_dfs[condition][0].columns)
    comp_dfs_cof[condition][0]['total wt reads rep2'] = comp_dfs_cof[condition][1]['total wt reads'] 
    comp_dfs_cof[condition][0]['total edited reads rep2'] = comp_dfs_cof[condition][1]['total edited reads'] 
    comp_dfs_cof[condition][0]['edited reads with indels rep2'] = comp_dfs_cof[condition][1]['edited reads with indels '] 
    comp_dfs_cof[condition][0]['wt reads with indels rep2'] = comp_dfs_cof[condition][1]['wt reads with indels'] 
    comp_dfs_cof[condition][0]['edited indel % rep2'] = comp_dfs_cof[condition][1]['edited indel %']
    comp_dfs_cof[condition][0]['wt indel % rep2'] = comp_dfs_cof[condition][1]['wt indel %'] 
    comp_dfs_cof[condition][0]['indel p-value rep2'] = comp_dfs_cof[condition][1]['indel p-value'] 

    #use combine_pvalues to combine two p-values that result from a fisher test
    combined_pvals = []
    for p1, p2 in zip(comp_dfs_cof[condition][0]['indel p-value'], comp_dfs_cof[condition][0]['indel p-value rep2']):
        stat, p_combined = combine_pvalues([p1, p2], method='fisher')
        combined_pvals.append(p_combined)

    #combine the p-values 
    comp_dfs_cof[condition][0]['fishers combined indel p-value'] = combined_pvals

    comp_dfs_cof[condition][0]['combined_edited_reads'] = comp_dfs_cof[condition][0]['total edited reads'] + comp_dfs_cof[condition][1]['total edited reads']
    comp_dfs_cof[condition][0]['combined_wt_reads'] = comp_dfs_cof[condition][0]['total wt reads'] + comp_dfs_cof[condition][1]['total wt reads']
    comp_dfs_cof[condition][0]['combined_edited_reads_w_indels'] = comp_dfs_cof[condition][0]['edited reads with indels '] + comp_dfs_cof[condition][1]['edited reads with indels ']
    comp_dfs_cof[condition][0]['combined_wt_reads_w_indels'] = comp_dfs_cof[condition][0]['wt reads with indels'] + comp_dfs_cof[condition][1]['wt reads with indels']
    comp_dfs_cof[condition][0]['combined edited indel %'] = comp_dfs_cof[condition][0]['combined_edited_reads_w_indels'] / comp_dfs_cof[condition][0]['combined_edited_reads']
    comp_dfs_cof[condition][0]['combined wt indel %'] = comp_dfs_cof[condition][0]['combined_wt_reads_w_indels'] / comp_dfs_cof[condition][0]['combined_wt_reads']

    #a different way of combining based off total reads - not used in final csv table
    comp_dfs_cof[condition][0]['fishers combined indel %'] = comp_dfs_cof[condition][0]['combined_wt_reads_w_indels'] / comp_dfs_cof[condition][0]['combined_wt_reads']


    #iterate through the rows in the comparison table and preform the fishers exact test on the combined data (not used in final csv)
    for index, row in comp_dfs_cof[condition][0].iterrows():
        region = row['REGION']
        cont = pd.DataFrame([])
        cont_indel = pd.DataFrame([])
        cont['wt'] = [int(row['combined_wt_reads_w_indels']),
                      int(row['combined_wt_reads']) - int(row['combined_wt_reads_w_indels'])]
        
        cont['edited'] = [int(row['combined_edited_reads_w_indels']),
                      int(row['combined_edited_reads']) - int(row['combined_edited_reads_w_indels'])]

        cont['type'] = ['indel', 'no indel']
        cont.set_index('type', inplace=True)
        odds_ratio, p_value = stats.fisher_exact(cont)
        comp_dfs_cof[condition][0].loc[comp_dfs_cof[condition][0]['REGION'] == region, 'combined indel p-value'] = p_value


#recreating the p-value calculation now that we include all locations
for condition in all_tsvs.keys():   
    comp_dfs[condition][0]['total wt reads rep2'] = comp_dfs[condition][1]['total wt reads'] 
    comp_dfs[condition][0]['total edited reads rep2'] = comp_dfs[condition][1]['total edited reads'] 
    comp_dfs[condition][0]['edited reads with indels rep2'] = comp_dfs[condition][1]['edited reads with indels '] 
    comp_dfs[condition][0]['wt reads with indels rep2'] = comp_dfs[condition][1]['wt reads with indels'] 
    comp_dfs[condition][0]['edited indel % rep2'] = comp_dfs[condition][1]['edited indel %'] 
    comp_dfs[condition][0]['indel p-value rep2'] = comp_dfs[condition][1]['indel p-value'] 
    comp_dfs[condition][0]['wt indel % rep2'] = comp_dfs[condition][1]['wt indel %'] 

    comp_dfs[condition][0]['combined_edited_reads'] = comp_dfs[condition][0]['total edited reads'] + comp_dfs[condition][1]['total edited reads']
    comp_dfs[condition][0]['combined_wt_reads'] = comp_dfs[condition][0]['total wt reads'] + comp_dfs[condition][1]['total wt reads']
    comp_dfs[condition][0]['combined_edited_reads_w_indels'] = comp_dfs[condition][0]['edited reads with indels '] + comp_dfs[condition][1]['edited reads with indels ']
    comp_dfs[condition][0]['combined_wt_reads_w_indels'] = comp_dfs[condition][0]['wt reads with indels'] + comp_dfs[condition][1]['wt reads with indels']
    comp_dfs[condition][0]['combined edited indel %'] = comp_dfs[condition][0]['combined_edited_reads_w_indels'] / comp_dfs[condition][0]['combined_edited_reads']
    comp_dfs[condition][0]['combined wt indel %'] = comp_dfs[condition][0]['combined_wt_reads_w_indels'] / comp_dfs[condition][0]['combined_wt_reads']

    combined_pvals = []
    for p1, p2 in zip(comp_dfs[condition][0]['indel p-value'], comp_dfs[condition][0]['indel p-value rep2']):
        stat, p_combined = combine_pvalues([p1, p2], method='fisher')
        combined_pvals.append(p_combined)
        
    comp_dfs[condition][0]['fishers combined indel p-value'] = combined_pvals
    
    #calculates the p-value for each of the rows
    for index, row in comp_dfs[condition][0].iterrows():
        region = row['REGION']
        cont = pd.DataFrame([])
        cont_indel = pd.DataFrame([])
        cont['wt'] = [int(row['combined_wt_reads_w_indels']),
                      int(row['combined_wt_reads']) - int(row['combined_wt_reads_w_indels'])]
        
        cont['edited'] = [int(row['combined_edited_reads_w_indels']),
                      int(row['combined_edited_reads']) - int(row['combined_edited_reads_w_indels'])]

        cont['type'] = ['indel', 'no indel']
        cont.set_index('type', inplace=True)
        odds_ratio, p_value = stats.fisher_exact(cont)
        comp_dfs[condition][0].loc[comp_dfs[condition][0]['REGION'] == region, 'combined indel p-value'] = p_value


In [ ]:
#add a row for the new table
def construct_comp_tbl(wt, edited):
    comp_df = pd.DataFrame([])

    # Ensure 'REGION' column is included
    comp_df['REGION'] = wt['REGION'].tolist()
    
    #construct a contigency table
    for index, row in wt.iterrows():
        cont = pd.DataFrame([])
        cont_indel = pd.DataFrame([])
        
        region = row['REGION']
        cont['wt'] = [int(row['combined_wt_reads_w_indels']),
        int(row['combined_wt_reads']) - int(row['combined_wt_reads_w_indels'])]

        row2 = edited.iloc[index]
        cont['edited'] = [int(row2['combined_edited_reads_w_indels']),
        int(row2['combined_edited_reads']) - int(row2['combined_edited_reads_w_indels'])]
        
        cont['type'] = ['indel', 'no indel']
        cont.set_index('type', inplace=True)
        odds_ratio, p_value = stats.fisher_exact(cont)

        comp_df.loc[comp_df['REGION'] == region, 'chrom'] = row['chrom']
        comp_df.loc[comp_df['REGION'] == region, 'start'] = row2['start']
        comp_df.loc[comp_df['REGION'] == region, 'end'] = row['end']
        comp_df.loc[comp_df['REGION'] == region, 'combined indel p-value'] = p_value
        
    return comp_df

In [ ]:
#understand the unique positions
for condition in comp_dfs:
    for replicate in np.arange(0,1):
        print(comp_dfs[condition][replicate]['chrom'].unique())

for condition in comp_dfs_cof:
    for replicate in np.arange(0,1):
        print(comp_dfs_cof[condition][replicate]['chrom'].unique())

In [ ]:
from statsmodels.stats.multitest import multipletests

#combine the two tables
for condition in all_tsvs:
    for replicate in range(1):  # Iterate through two replicates (0 and 1)
        comp_dfs_cof[condition][replicate]['category'] = 'Cas-OFFinder'
        comp_dfs[condition][replicate]['category'] = 'CHANGE-seq'
        comp_dfs[condition][replicate]=pd.concat([comp_dfs_cof[condition][replicate],comp_dfs[condition][replicate]])


#aggregate the 
for condition in ['ALT7', 'ALTC', 'REXC', 'REX4']:
    for replicate in [0, 1]:
        df = comp_dfs[condition][replicate]

        agg_dict = {}
        for col in df.columns:
            if col == 'start':
                continue  # grouping column
            elif col == 'category':
                # Join all category values (as strings)
                agg_dict[col] = lambda x: ' and '.join(x.astype(str))
            else:
                # Take the first value for any other column
                agg_dict[col] = 'first'

        comp_dfs[condition][replicate] = df.groupby('start').agg(agg_dict).reset_index()

for condition in all_tsvs:
    for replicate in range(0,1):  
        # Get the raw p-values for the current replicate and condition
        raw_p_values = comp_dfs[condition][replicate]['indel p-value']
        combined_raw_p_values = comp_dfs[condition][replicate]['combined indel p-value']

        # Combine based off fishers combined p-value calculation
        fishers_combined_raw_p_values = comp_dfs[condition][replicate]['fishers combined indel p-value']
        
        # apply Benjamini-Hochberg correction (FDR control) to the fishers combined p-value
        reject, fisher_adjusted_p_values, _, _ = multipletests(fishers_combined_raw_p_values, alpha=0.05, method='fdr_bh')
        
        # Apply Benjamini-Hochberg correction (FDR control)
        reject, adjusted_p_values, _, _ = multipletests(combined_raw_p_values, alpha=0.05, method='fdr_bh')
        combined_reject, combined_adjusted_p_values, combined_, combined_ = multipletests(combined_raw_p_values, alpha=0.05, method='fdr_bh')
        
        # Add the adjusted p-values back into the DataFrame (not used)
        comp_dfs[condition][replicate]['adjusted_raw_p_values'] = adjusted_p_values
        comp_dfs[condition][replicate]['combined adjusted indel p-value'] = combined_adjusted_p_values

        #final p-value metric to use
        comp_dfs[condition][replicate]['fishers p-values adjusted with benjamini-hochberg'] = fisher_adjusted_p_values


for condition in ['ALT7', 'ALTC', 'REXC', 'REX4']:
    for replicate in [0, 1]:
        comp_dfs[condition][replicate].to_csv(f"{condition}{replicate}_combined_original_new.csv", sep='\t', index=False)



In [ ]:
comp_dfs['ALTC'][0]['category'].unique()

In [ ]:
min(comp_dfs['ALTC'][0]['fishers combined indel p-value']) #correct
min(multipletests(comp_dfs['ALTC'][0]['fishers combined indel p-value'], alpha=0.05, method='fdr_bh')[1]) #correct
min(comp_dfs['ALTC'][0]['fishers p-values adjusted with benjamini-hochberg'].to_list()) #not correct


fishers_combined_raw_p_values = comp_dfs['ALTC'][0]['fishers combined indel p-value']
min(fishers_combined_raw_p_values) #correct
reject, fisher_adjusted_p_values, _, _ = multipletests(fishers_combined_raw_p_values, alpha=0.05, method='fdr_bh')
min(fisher_adjusted_p_values) #correct
min(comp_dfs['ALTC'][0]['fishers combined indel p-value']) #correct
min(comp_dfs['ALTC'][0]['fishers combined indel p-value']) #correct
min(comp_dfs['ALTC'][0]['fishers p-values adjusted with benjamini-hochberg'].to_list()) #not correct

In [ ]:

# Define the chrom_order list - removing chromosome Y for this analysis as the c9p5 cell line does not contain chrom Y
chrom_order = ['chr1', 'chr2', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 'chr8', 
               'chr9', 'chr10', 'chr11', 'chr12', 'chr13', 'chr14', 'chr15', 
               'chr16', 'chr17', 'chr18', 'chr19', 'chr20', 'chr21', 'chr22', 'chrX']

# Loop through each condition and replicate
for condition in all_tsvs:
    for replicate in np.arange(0, 2):
        print(condition, replicate)
        # Read the DataFrame (assuming file_path is defined earlier)
        
        # Filter the DataFrame to include only rows with 'chrom' values in chrom_order
        filtered_df = comp_dfs[condition][replicate][comp_dfs[condition][replicate]['chrom'].isin(chrom_order)]
        
        # Sort the DataFrame by 'chrom' according to the chrom_order list
        filtered_df['chrom'] = pd.Categorical(filtered_df['chrom'], categories=chrom_order, ordered=True)
        sorted_df = filtered_df.sort_values(by=['chrom', 'start'])
        
        # Update the DataFrame with the filtered and sorted data
        comp_dfs[condition][replicate] = sorted_df

        # If you want to display the sorted DataFrame
        display(sorted_df)


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np


# Define alternating black-gray colors
black_gray = ['#000000', '#808080']  # Black and Gray

# # Generate dictionary with alternating colors
# chromosomes = [f'chr{i}' for i in range(1, 23)]
# custom_palette = {chrom: black_gray[i % 2] for i, chrom in enumerate(chromosomes)}
# custom_palette['chrX'] = '#000000'
# custom_palette['chrY'] = '#808080'
# #color based off location


#color based of  what its located in
category = ['CHANGE-seq', 'Cas-OFFinder and CHANGE-seq', 'Cas-OFFinder']
ordered_labels = ['CHANGE-seq', 'Cas-OFFinder', 'Cas-OFFinder and CHANGE-seq']
desired_order = ['CHANGE-seq', 'Cas-OFFinder and CHANGE-seq', 'Cas-OFFinder']
custom_palette = {'CHANGE-seq': '#1b9e77',  'Cas-OFFinder': '#d95f02', 'Cas-OFFinder and CHANGE-seq': '#46436b'}

# Print to verify
print(custom_palette)


#need to modify to include the chromosome X and the chromsome Y
#make the y axis the adjusted p-value instead

count=0
for condition in all_tsvs:
    for replicate in np.arange(0,1):
        # Create the data - select the columns of interest
        data = {
            'chrom': comp_dfs[condition][replicate]['chrom_orig'],
            'category': comp_dfs[condition][replicate]['category'],
            'start': comp_dfs[condition][replicate]['start_orig'],
            'end': comp_dfs[condition][replicate]['end_orig'],
            'p-value': comp_dfs[condition][replicate]['fishers p-values adjusted with benjamini-hochberg']
        }
        
        df = pd.DataFrame(data)
        df['%edited difference'] = comp_dfs[condition][replicate]['edited indel %'] - comp_dfs[condition][replicate]['wt indel %']

        start_offset = []

        # Initialize the dictionary for offsets
        offsets = {}
        
        # Start by initializing the current position
        current_pos = 0
        
        # Iterate through each unique chromosome
        for chrom in df['chrom'].unique():            
            # Get the data for the current chromosome
            chrom_data = df[df['chrom'] == chrom]
            
            # Find the maximum start position for the chromosome (end position)
            chrom_end = chrom_data['start'].max()  # Max position = end position for this chromosome
            
            # Store the offset (end position of the chromosome) in the dictionary
            offsets[chrom] = current_pos  # Offset is the cumulative position
            
            # Update the current position for the next chromosome
            current_pos += chrom_end  # Add the current chromosome's end position to the cumulative position
        
        # Print the offsets dictionary to see the results
        df['offset_start'] = df.apply(lambda row: row['start'] + offsets[row['chrom']], axis=1)
 
        # Transform p-values to -log10(p-value)
        df['-log10 p-value'] = -np.log10(df['p-value'])
        
        # Plot the Manhattan plot
        plt.figure(figsize=(13, 8))
        
        # Create scatter plot and remove edgecolor (no outline for dots)
        sns.scatterplot(data=df, x="offset_start", y="-log10 p-value", hue="category", palette=custom_palette, alpha=0.8, edgecolor='none', s = 300)
            
        # Label the plot
        plt.xlabel("Chromosome", fontsize=30,  fontfamily='Arial')
        plt.ylabel("–log10(p), Edited vs. Control Indel Rate", fontsize=30,  fontfamily='Arial')
        if condition == 'ALT7':
            title_name = "7'"
        if condition == 'ALTC':
            title_name = "C'"
        if condition == 'REXC':
            title_name = "C"
        if condition == 'REX4':
            title_name = '4'
        plt.title('On and Off-Targets detected by WGS for gRNA ' + title_name, fontsize=25,  fontfamily='Arial')
        
        # Customize the X-axis to display chromosomes properly (and adjust spacing if needed)
        chrom_boundaries = []
        chrom_labels = []
        
        # Track the cumulative position for each chromosome
        current_pos = 0
        for chrom in df['chrom'].unique():
            chrom_data = df[df['chrom'] == chrom]
            chrom_end = chrom_data['start'].max()  # Get the max position for the chromosome
            chrom_boundaries.append(current_pos)  # Add the offset
            chrom_labels.append(chrom)  # Add the chrom label
            current_pos += chrom_end  # Update current position

        #find which points are signficant
        significant_df = df[df['p-value'] < 0.05]
        
        # Label significant points
        for _, row in significant_df.iterrows():
            if row['chrom'] == 'chr9':
                plt.text(row['offset_start'] - 10, row['-log10 p-value'] + 10, 
                     f"On-Target {row['chrom']}:{int(row['start'])}:{int(row['end'])}", fontsize=25, ha='right', color='black',  fontfamily='Arial')
            else:
                 plt.text(row['offset_start'] - 10, row['-log10 p-value'] + 10, 
                     f"Off-Target {row['chrom']}:{int(row['start'])}:{int(row['end'])}", fontsize=25, ha='right', color='black',  fontfamily='Arial')

        # Set custom x-ticks based on chromosome boundaries
        plt.tick_params(axis='x', width=2)
        plt.tick_params(axis='y', width=2)
        plt.xticks(ticks=chrom_boundaries, labels=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,'X'], rotation=90,fontsize=30)
        plt.tick_params(axis='x', length=12)
        plt.tick_params(axis='y', length=12)
        plt.yticks(fontsize=30)

        
        # Add a legend to indicate the chromosomes
        handles, labels = plt.gca().get_legend_handles_labels()
        label_to_handle = dict(zip(labels, handles))
        
        ordered_labels = ['CHANGE-seq', 'Cas-OFFinder', 'Cas-OFFinder and CHANGE-seq']
        ordered_handles = [label_to_handle[label] for label in ordered_labels if label in label_to_handle]
        
        legend = plt.legend(ordered_handles, ordered_labels, title="Nomination Method", bbox_to_anchor=(0.63, 1), loc='upper left', fontsize='x-large')
        legend.get_title().set_fontsize('x-large') 

        plt.ylim(-10,230)
            
        # Show the plot
        count += 1
        plt.axhline(y=3, color='blue', linestyle=':', linewidth=2)
        plt.tight_layout()
        # Increment the count
        
        # Save the plot with the count in the filename
        plt.savefig(f"Final_Plot_new_again{count}.svg", format='svg')  # <-- Use f-string to evaluate count
        plt.show()


In [ ]:
output_dir = "filtered_tables"
os.makedirs(output_dir, exist_ok=True)

excel_path = os.path.join(output_dir, "WGS_pvalues_fishers_2_replicates_result.xlsx")


#produce the final tables that contain only all the columns of interest
cats = {'ALTC': 'grna_c_prime', 'ALT7':'grna_7_prime', 'REX4': 'grna_4', 'REXC':'grna_c'}
with pd.ExcelWriter(excel_path, engine="openpyxl") as writer:
    for condition in all_tsvs:
        for replicate in np.arange(0, 1):
            df = comp_dfs[condition][replicate].copy()
            
            #irrelevant columns to drop for final table
            cols_to_drop = ['adjusted indel p-value',
                            'combined indel p-value', 'combined_edited_reads', 'combined_wt_reads',
                            'combined edited indel %', 'combined wt indel %', 
                            'combined_edited_reads_w_indels', 
                            'combined_wt_reads_w_indels', 
                            'raw_p_values',  'Unnamed: 0', 'REGION',
                            'combined adjusted indel p-value', 'adjusted_raw_p_values', 
                            'fishers combined indel %', 'chrom_orig', 'start_orig', 'end_orig', 'trial']
            df.drop([c for c in cols_to_drop if c in df.columns], axis=1, inplace=True)

            # Sheet name: e.g., "cond1_rep0"
            sheet_name = f"{cats[condition]}"[:31]  # Excel limit = 31 chars
            df.to_excel(writer, sheet_name=sheet_name, index=False)
